## Step 1: Setup

In [7]:
#@title Setup
!git clone https://github.com/oneThousand1000/HairMapper
import os
os.chdir('./HairMapper')

# ==================== download checkpoint ====================
!pip install gdown
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

print('Please wait util all models are downloaded...')


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def download_from_google_drive(file_id,file_dst):
  downloaded = drive.CreateFile({'id':file_id})
  downloaded.FetchMetadata(fetch_all=True)
  downloaded.GetContentFile(file_dst)

checkpoints ={
    'StyleGAN2-ada-Generator.pth':
    {
        'url':'1EsGehuEdY4z4t21o2LgW2dSsyN3rxYLJ',
        'dir': './ckpts'
    },
    'e4e_ffhq_encode.pt':
    {
        'url':'1cUv_reLE6k3604or78EranS7XzuVMWeO',
        'dir': './ckpts'
    },
    'model_ir_se50.pth':
    {
        'url':'1GIMopzrt2GE_4PG-_YxmVqTQEiaqu5L6',
        'dir': './ckpts'
    },
    'face_parsing.pth':
    {
        'url':'1IMsrkXA9NuCEy1ij8c8o6wCrAxkmjNPZ',
        'dir': './ckpts'
    },
    'vgg16.pth':
    {
        'url':'1EPhkEP_1O7ZVk66aBeKoFqf3xiM4BHH8',
        'dir': './ckpts'
    }
}
for ckpt in checkpoints:
  name = ckpt
  url = checkpoints[name]['url']
  output_dir = checkpoints[name]['dir']
  os.makedirs(output_dir,exist_ok=True)
  output_path = os.path.join(output_dir,name)
  #gdown.download(url=url,output=output_path,quiet=False) # bug
  download_from_google_drive(file_id=url, file_dst=output_path)

classification_ckpt =[
        {'url':'1SSw6vd-25OGnLAE0kuA-_VHabxlsdLXL',
        'dir': './classifier/gender_classification'},
        {'url':'1n14ckDcgiy7eu-e9XZhqQYb5025PjSpV',
        'dir': './classifier/hair_classification'}
]
for clf_ckpt_dict in classification_ckpt:
  name = 'classification_model.pth'
  url = clf_ckpt_dict['url']
  output_dir = clf_ckpt_dict['dir']
  os.makedirs(output_dir,exist_ok=True)
  output_path = os.path.join(output_dir,name)
  #gdown.download(url=url,output=output_path,quiet=False)
  download_from_google_drive(file_id=url, file_dst=output_path)





Cloning into 'HairMapper'...
remote: Enumerating objects: 415, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 415 (delta 21), reused 18 (delta 18), pack-reused 373 (from 1)
Receiving objects: 100% (415/415), 38.38 MiB | 17.23 MiB/s, done.
Resolving deltas: 100% (81/81), done.
Please wait util all models are downloaded...


In [8]:
# ==================== install packages ====================
!pip install torch===2.0.0+cu117 torchvision===0.15.0+cu117 torchaudio===2.0.0+cu117 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -r requirements.txt
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 449.9 kB/s eta 0:00:00
  Using cached https://download.pytorch.org/whl/cu117/torchvision-0.15.0%2Bcu117-cp311-cp311-linux_x86_64.whl (6.1 MB)
  Using cached https://download.pytorch.org/whl/cu117/torchaudio-2.0.0%2Bcu117-cp311-cp311-linux_x86_64.whl (4.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0+cu124
    Uninstalling torch-2.6.0+cu124:
      Successfully uninstalled torch-2.6.0+cu124
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.21.0+cu124
    Uninstalling torchvision-0.21.0+cu124:
   

## Step 2: pre-trained models access

Please fill out this form for **pre-trained models access**: https://forms.gle/a5pRbE3yxEr7sZDm7

Then fill out the mapper_url with the url of [Final HairMapper]



In [13]:
mapper_url = 'https://drive.google.com/file/d/1F3oujXbvalqEOixcAkIyURuY512nmroe' #@param {type:"string"}

## Step 3: download pre-trained model and test image

In [14]:
#@title Download Pretrain Models

name = 'best_model.pt'
mapper_url = mapper_url.replace('https://drive.google.com/file/d/','')
output_dir = './mapper/checkpoints/final'
os.makedirs(output_dir,exist_ok=True)
output_path = os.path.join(output_dir,name)
download_from_google_drive(file_id=mapper_url, file_dst=output_path)


In [16]:
#@title Download test image
name = '00006.png'
url = '1UlROI0W8sPXUbufzQ0cm8yi9dzX9V6vd'
output_dir = './test_data/origin'
os.makedirs(output_dir,exist_ok=True)
output_path = os.path.join(output_dir,name)
download_from_google_drive(file_id=url, file_dst=output_path)

## Step 4: Run encoder

In [17]:
os.chdir('./encoder4editing')

Please put the real images to **./test_data/origin** (examplar data can be found in ./test_data/origin/00006.png).



In [19]:
!pip install "numpy<2.0"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 99.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [20]:
from argparse import Namespace
import sys
import torch
import torchvision.transforms as transforms
import numpy as np
import PIL.Image
from PIL import ImageFile
import glob
import os
import argparse
sys.path.append(".")
sys.path.append("..")
ImageFile.LOAD_TRUNCATED_IMAGES = True
from models.psp import pSp

In [21]:
def run_on_batch(inputs, net):
    latents = net(inputs.to("cuda").float(), randomize_noise=False, return_latents=True)
    return latents


In [22]:
img_transforms = transforms.Compose([
          transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])]
            )
model_path = "../ckpts/e4e_ffhq_encode.pt"
ckpt = torch.load(model_path, map_location='cpu')
opts = ckpt['opts']
opts['checkpoint_path'] = model_path
opts= Namespace(**opts)
net = pSp(opts)
net.eval()
net.cuda()

Loading e4e over the pSp framework from checkpoint: ../ckpts/e4e_ffhq_encode.pt


pSp(
  (encoder): Encoder4Editing(
    (input_layer): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=64)
    )
    (body): Sequential(
      (0): bottleneck_IR_SE(
        (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
        (res_layer): Sequential(
          (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (2): PReLU(num_parameters=64)
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): SEModule(
            (avg_pool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(64

In [23]:
data_dir = '../test_data'
file_dir = os.path.join(data_dir,'origin')
code_dir = os.path.join(data_dir,'code')
if not os.path.exists(code_dir):
    os.mkdir(code_dir)
for file_path in glob.glob(os.path.join(file_dir,'*.png'))+glob.glob(os.path.join(file_dir,'*.jpg')):
    name = os.path.basename(file_path)[:-4]
    code_path =os.path.join(code_dir,f'{name}.npy')
    if os.path.exists(code_path):
        continue
    input_image = PIL.Image.open(file_path)
    transformed_image = img_transforms(input_image)
    with torch.no_grad():
        latents = run_on_batch(transformed_image.unsqueeze(0), net)
        latent = latents[0].cpu().numpy()
        latent = np.reshape(latent,(1,18,512))
        np.save(code_path,latent)
        print(f'save to {code_path}')


RuntimeError: Numpy is not available

## Step 5: Run HairMaper

In [ ]:
os.chdir('../')

In [ ]:
import os.path
import argparse
import cv2
from styleGAN2_ada_model.stylegan2_ada_generator import StyleGAN2adaGenerator
from tqdm import tqdm
from classifier.src.feature_extractor.hair_mask_extractor import get_hair_mask, get_parsingNet
from mapper.networks.level_mapper import LevelMapper
import torch
import glob
from diffuse.inverter_remove_hair import InverterRemoveHair
import numpy as np
from PIL import ImageFile
import os
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
model_name = 'stylegan2_ada'
latent_space_type = 'wp'
data_dir = './test_data'
print(f'Initializing generator.')
model = StyleGAN2adaGenerator(model_name, logger=None, truncation_psi=1.0)

mapper = LevelMapper(input_dim=512).eval().cuda()
ckpt = torch.load('./mapper/checkpoints/final/best_model.pt')
alpha = float(ckpt['alpha']) * 1.2
mapper.load_state_dict(ckpt['state_dict'], strict=True)
kwargs = {'latent_space_type': latent_space_type}
parsingNet = get_parsingNet(save_pth='./ckpts/face_parsing.pth')
inverter = InverterRemoveHair(
        model_name,
        Generator=model,
        learning_rate=0.01,
        reconstruction_loss_weight=1.0,
        perceptual_loss_weight=5e-5,
        truncation_psi=1.0,
        logger=None
)

code_dir = os.path.join(data_dir, 'code')
origin_img_dir = os.path.join(data_dir, 'origin')
res_dir = os.path.join(data_dir, 'mapper_res')

os.makedirs(res_dir, exist_ok=True)

In [ ]:
code_list = glob.glob(os.path.join(code_dir, '*.npy'))

total_num = len(code_list)

print(f'Editing {total_num} samples.')
pbar = tqdm(total=total_num)
for index in range(total_num):
    pbar.update(1)
    code_path = code_list[index]
    name = os.path.basename(code_path)[:-4]
    f_path_png = os.path.join(origin_img_dir, f'{name}.png')
    f_path_jpg = os.path.join(origin_img_dir, f'{name}.jpg')
    if os.path.exists(os.path.join(res_dir, f'{name}.png')):
        continue
    if os.path.exists(f_path_png):
        origin_img_path = f_path_png
    elif os.path.exists(f_path_jpg):
        origin_img_path = f_path_jpg
    else:
        continue

    latent_codes_origin = np.reshape(np.load(code_path), (1, 18, 512))

    mapper_input = latent_codes_origin.copy()
    mapper_input_tensor = torch.from_numpy(mapper_input).cuda().float()
    edited_latent_codes = latent_codes_origin
    edited_latent_codes[:, :8, :] += alpha * mapper(mapper_input_tensor).to('cpu').detach().numpy()

    origin_img = cv2.imread(origin_img_path)

    outputs = model.easy_style_mixing(latent_codes=edited_latent_codes,
                                      style_range=range(7, 18),
                                      style_codes=latent_codes_origin,
                                      mix_ratio=0.8,
                                      **kwargs
                                      )

    edited_img = outputs['image'][0][:, :, ::-1]

    # --remain_ear: preserve the ears in the original input image.
    hair_mask = get_hair_mask(img_path=origin_img, net=parsingNet, include_hat=True, include_ear=True)

    mask_dilate = cv2.dilate(hair_mask,
                             kernel=np.ones((50, 50), np.uint8))
    mask_dilate_blur = cv2.blur(mask_dilate, ksize=(30, 30))
    mask_dilate_blur = (hair_mask + (255 - hair_mask) / 255 * mask_dilate_blur).astype(np.uint8)

    face_mask = 255 - mask_dilate_blur

    index = np.where(face_mask > 0)
    cy = (np.min(index[0]) + np.max(index[0])) // 2
    cx = (np.min(index[1]) + np.max(index[1])) // 2
    center = (cx, cy)

    res_save_path = os.path.join(res_dir, f'{name}.png')

    mixed_clone = cv2.seamlessClone(origin_img, edited_img, face_mask[:, :, 0], center, cv2.NORMAL_CLONE)


    cv2.imwrite(res_save_path, mixed_clone)


## Step 6: visualize results

In [ ]:
from IPython.display import display
import glob
import cv2
import numpy as np
from PIL import Image

In [ ]:
for res_path in glob.glob('./test_data/mapper_res/*'):
    res_img = cv2.imread(res_path)[:,:,::-1]
    input_path = res_path.replace('mapper_res','origin')
    input_img = cv2.imread(input_path)[:,:,::-1]
    visualize = np.concatenate([res_img,input_img], axis=1)
    res_im = Image.fromarray(visualize)
    display(res_im)